# 0. 한나눔 형태소 분석기: 띄어쓰기되지 않은 문장에 대해서 문제 발생

한나눔의 경우, 한번에 처리되는 텍스트의 범주에 한계가 있었음.

약 1000개까지 정상적으로 토큰화가 됨을 확인하여, 1000보다 사이즈가 훨씬 큰 텍스트 문서를 약 1000문자단위씩 쪼개어 처리함

In [ ]:
data = pd.read_excel('preprocessing_cnts_dropna_data/preprocess_dropna_2015_cafe_ssal.xlsx')

In [ ]:
#데이터 타입을 모두 str 로 저장
data = data.astype('str')

In [ ]:
# 사이즈 줄여 str로 저장 960번 나눠서 저장 (10*960번) 960번을 더할것 즉, 10개묶음 960개 만들기
for i in range(0,960): # 960개 : text0~959
    globals()['text{}'.format(i)] = " " # 1000개 text변수 생성
    for j in range(10*i,10*(i+1) ):
        globals()['text{}'.format(i)] += data['content'][j] # 1000개의 text변수에 10개 묶음씩 저장
        # 0~9 / 10~19 / 20~29 저장~~ 9600~9609

# 마지막 데이터 수동 저장
text960 = ' '
for k in range(9600,9607): # 9606번까지 존재
    text960 += data['content'][k]

In [ ]:
# 데이터 정제
import re
for i in range(0,961): # text0~960
    globals()['text{}'.format(i)] = re.sub('[0-9]+', '', globals()['text{}'.format(i)]) # 숫자 제거
    globals()['text{}'.format(i)] = re.sub('[A-Za-z]+', '', globals()['text{}'.format(i)]) # 알파벡 제거
    globals()['text{}'.format(i)] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》]', '', globals()['text{}'.format(i)]) # 특수문자 제거

In [ ]:
hannanum = Hannanum()
for i in range(0,961):
    globals()['tokenized_text{}'.format(i)] = hannanum.nouns(globals()['text{}'.format(i)] ) # data['content'][2]
tokenized_text0

In [ ]:
# 판다스 시리즈 만들기
for i in range(0,961):
    globals()['tokenized_texts{}'.format(i)] =pd.Series(globals()['tokenized_text{}'.format(i)])

In [ ]:
# 시리즈 합치기
series_list = []
for i in range(0,961):
    series_list.append(globals()['tokenized_texts{}'.format(i)] )
tokenized_texts_series = pd.concat(series_list)

In [ ]:
# 토큰화된 단어 전체 빈도와 함께 확인
whole_word_series = tokenized_texts_series.value_counts()

In [ ]:
# 데이터 저장
whole_word_series.to_excel('word_series_2015_cafe.xlsx',encoding='utf-8')

### 띄어쓰기가 잘 처리되지 않았던 hannanum에 네이버 맞춤법검사기 이용한 라이브러리 hanspell 이용해보자!

# 1. 띄어쓰기 처리 py.hanspell 이용

In [3]:
from hanspell import spell_checker

import pandas as pd

In [7]:
data = pd.read_excel('preprocessing_cnts_dropna_data/preprocess_dropna_2015_cafe_ssal.xlsx')

In [25]:
data = data.astype('str')

In [26]:
data['content']

0       세종시로 올해 1월 10일 이사했습니다.어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구...
1       저희 큰아버지께서 쌀도매를 하세요^^빨간쌀 이예요.저도 작녁부터 계속 먹고있는데요^...
2                                     제가 변비가 심해서 현미쌀100%를
3       그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요 아기간식도 요즘 고민...
4       심심한 일요일 오후.당근쌀머핀 만들어 보았어요~당근이랑 우유는 휘리릭 갈아주고밀가루...
                              ...                        
9602    요리전용맛술 마트가면 보기는 했는데 저게 꼭 필요할까 하는 생각에 구매를 망설였는데...
9603                           가능한가요;;;;;믹서기 청소가 귀찮아서 ;;;
9604    담주부터 초기이유식 시작인데 아직 애가 못앉잖아요. 다들 그냥 분유주듯 안아서 먹이...
9605                 어뭉들어디서 사드시나요? 아이간식사먹이는 사이트추천좀해주셔요^ ^
9606    간혹 동물 사료용으로 쓰신다고 무료나눔하신던데...벌레가 막 나오거나 하지는 않는데...
Name: content, Length: 9607, dtype: object

In [35]:
# 데이터 한개만 우선확인: data['content'][1]
result = spell_checker.check(data['content'][1])
result.as_dict()  # 딕셔너리로 확인

{'result': True,
 'original': '저희 큰아버지께서 쌀도매를 하세요^^빨간쌀 이예요.저도 작녁부터 계속 먹고있는데요^^검정쌀처럼 찹쌀적미라 밥에 섞어 드시는건데요.항암효과.당뇨예방.노화방지~~빨간색성분은 몸에 좋다고들 하잖아요.저 아시는 지인분들께서 구매를 원하셔서 오늘 시골왔다가 내일 올라가는데요^^이번에 새로농사지은 햇찹쌀이예요♡2.5키로에 15000원이구요.시중에서는 보기힘든 귀한쌀이구요.백화점에서는1키로에 1만원에 판매되고 있는 비싼쌀이예요.필요하신분들~댓글주세요♡제가 차로 가지고 가는거라~~많이는 못가져가요^^;;\xa0\xa0\xa0\xa0',
 'checked': '저희 큰아버지께서 쌀도 애를 하세요^^빨간 쌀 이예요. 저도 작녁부터 계속 먹고 있는데요^^검정쌀처럼 찹쌀적 미라 밥에 섞어 드시는 건데요. 항암효과. 당뇨 예방. 노화 방지~~빨간색 성분은 몸에 좋다고들 하잖아요. 저 아시는 지인분들께서 구매를 원하셔서 오늘 시골 왔다가 내일 올라가는데요^^이번에 새로 농사지은 햇찹쌀이에요♡2.5킬로에 15000원이고요. 시중에서는 보기 힘든 귀한 쌀이고요. 백화점에서는 1킬로에 1만 원에 판매되고 있는 비싼 쌀이에요. 필요하신 분들~댓글 주세요♡제가 차로 가지고 가는 거라~~많이는 못 가져가요^^;;\xa0\xa0\xa0\xa0',
 'errors': 18,
 'words': OrderedDict([('저희', 0),
              ('큰아버지께서', 0),
              ('쌀도', 4),
              ('애를', 4),
              ('하세요^^빨간', 2),
              ('쌀', 2),
              ('이예요.', 2),
              ('저도', 2),
              ('<span', 0),
              ("class='violet_text'>작녁부터", 0),
              ('계속', 0),
     

In [45]:
# 딕셔너리에서 checked값만 확인
result.checked # 맞춤법 교정된 str만

'저희 큰아버지께서 쌀도 애를 하세요^^빨간 쌀 이예요. 저도 작녁부터 계속 먹고 있는데요^^검정쌀처럼 찹쌀적 미라 밥에 섞어 드시는 건데요. 항암효과. 당뇨 예방. 노화 방지~~빨간색 성분은 몸에 좋다고들 하잖아요. 저 아시는 지인분들께서 구매를 원하셔서 오늘 시골 왔다가 내일 올라가는데요^^이번에 새로 농사지은 햇찹쌀이에요♡2.5킬로에 15000원이고요. 시중에서는 보기 힘든 귀한 쌀이고요. 백화점에서는 1킬로에 1만 원에 판매되고 있는 비싼 쌀이에요. 필요하신 분들~댓글 주세요♡제가 차로 가지고 가는 거라~~많이는 못 가져가요^^;;\xa0\xa0\xa0\xa0'

## 01. hanspell에서 500자 이상 인식 못하기 때문에 str로 한번에 넣어 500자씩 끊기
texts500_i (where i = 0~3086)으로 저장

In [83]:
# 한 블럭에 모든 문장 넣음
text500s = ""
for i in range(len(data)):
    text500s += data['content'][i]

In [97]:
# 데이터 정제
import re
text500s = re.sub('[A-Za-z]+', '',text500s) # 영어 제거
text500s = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》]', '', text500s) # 특수문자 제거

In [102]:
text500s[1543239] #맨 마지막 문자 확인

'요'

In [2]:
1543239 /500, 1543239 - 3086*500 # 반복할 횟수와, 맨마지막에 저장할 개수 확인

(3086.478, 239)

In [130]:
# 사이즈 줄여 str로 저장 3086번 나눠서 저장 (500*3086번) 3086번을 더할것 즉, 500개묶음 3086개 만들기
for i in range(0,3086): # 3086개 : text0~3085
    globals()['text500_{}'.format(i)] = '' # 3086개 text변수 생성
    for j in range(500*i,500*(i+1) ):
        globals()['text500_{}'.format(i)] += text500s[j] # 3086개의 text변수에 500개 묶음씩 저장
        # 0~499 / 500~999 / 1000~1499 저장~~ 

# 마지막 데이터3086 수동 저장
text500_3086 = ' '
for k in range(1543000,1543240): # 1543239번까지 존재
    text500_3086 +=text500s[k]

#text500_0~ 3086 데이터 분리 완성

In [131]:
text500_3086 # 데이터 확인

' 아주고 감칠맛 까지 있네요멸치도 생선이니ㅋㅋㅋ다음에는 캠핑장에가서 고기 구워먹을때 사용해 봐야 겠여요가능한가요;;;;;믹서기 청소가 귀찮아서 ;;;담주부터 초기이유식 시작인데 아직 애가 못앉잖아요 다들 그냥 분유주듯 안아서 먹이시나요 어뭉들어디서 사드시나요 아이간식사먹이는 사이트추천좀해주셔요 간혹 동물 사료용으로 쓰신다고 무료나눔하신던데벌레가 막 나오거나 하지는 않는데 애기들이랑 밥해먹기에는 좀 \xa0그래서요큰봉지로 한봉지정도되요'

## 02. text500_i 데이터에 hanspell 적용해보자!

In [134]:
dict_list=[]
for i in range(3087): # 0~3086까지 존재
    globals()['result{}'.format(i)] = spell_checker.check(globals()['text500_{}'.format(i)] )
    globals()['result{}'.format(i)].as_dict()
    globals()['spelling_done{}'.format(i)] = globals()['result{}'.format(i)].checked 


In [137]:
spelling_done=[]
for i in range(3087):
    spelling_done.append(globals()['spelling_done{}'.format(i)] )  #spelling맞춤법 검사완료된 데이터 저장

# 2. 빈도분석 

## 01. 한나눔 형태소 분석기 이용

In [140]:
from konlpy.tag import *

In [141]:
hannanum = Hannanum()
for i in range(0,3087):
    globals()['tokenized_text{}'.format(i)] = hannanum.nouns( globals()['spelling_done{}'.format(i)] ) # 한나눔도 한번에 많은 양의 텍스트 처리 불가! 나누어서 처리해 따로 저장
tokenized_text1

['15000원',
 '시중',
 '쌀',
 '백화점',
 '1킬로',
 '1',
 '원',
 '판매',
 '쌀',
 '필요',
 '들',
 '댓글',
 '주세요♡제',
 '차',
 '것',
 '가져가요;;\xa0\xa0\xa0\xa0제',
 '변비',
 '현미쌀',
 '100',
 '그동안',
 '공구',
 '후',
 '공구',
 '게시판',
 '찐빵',
 '구입',
 '아기',
 '식',
 '고민',
 '차',
 '백',
 '프로',
 '쌀',
 '찐빵',
 '이거',
 '후',
 '대',
 '안내',
 '대',
 '찜기',
 '프라이팬',
 '넘넘',
 '방법',
 '훈제',
 '고구마',
 '훈제',
 '찐빵',
 '생각',
 '^ㅎ^ㅎ그랬더니',
 '찜기',
 '프라이팬',
 '두',
 '가지',
 '찐빵',
 '겉',
 '바삭하',
 '속',
 '포근',
 '촉촉ㅎㅎㅎ넘넘',
 '올겨울',
 '마을',
 '찐빵',
 '함께\xa0\xa0\xa0\xa0심심한',
 '일요일',
 '오후',
 '당근',
 '머핀',
 '당근',
 '우유',
 '휘리릭',
 '밀가루',
 '대',
 '쌀가루',
 '오늘',
 '버터',
 '부푸러가루베이킹파우더',
 '걸',
 '토핑',
 '땅콩분',
 '때',
 '감말랭이다',
 '리',
 '뽕냥하']

## 02. 빈도 파악

In [142]:
# 판다스 시리즈 만들기
for i in range(0,3087):
    globals()['tokenized_texts{}'.format(i)] =pd.Series(globals()['tokenized_text{}'.format(i)])

<ipython-input-142-23b25d3150b0>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  globals()['tokenized_texts{}'.format(i)] =pd.Series(globals()['tokenized_text{}'.format(i)])


In [143]:
# 시리즈 합치기
series_list = []
for i in range(0,961):
    series_list.append(globals()['tokenized_texts{}'.format(i)] )
tokenized_texts_series = pd.concat(series_list)

In [144]:
# 판다스 시리즈 이용
result= tokenized_texts_series.value_counts().head(30)
result

것      2022
쌀      1777
분       684
이유식     674
쌀가루     589
수       586
밥       550
저       539
때       527
정도      447
데       446
거       363
집       354
시작      346
오늘      343
미음      317
맛       317
전       308
아기      288
원       275
후       274
번       257
중       245
두       244
지       230
양       223
곳       220
물       203
생각      197
나       195
dtype: int64

## 03. 전체 빈도 확인을 위해 엑셀로 저장

In [145]:
whole_word_series = tokenized_texts_series.value_counts()

In [147]:
whole_word_series .to_excel('hanspell_word_series_2015_cafe.xlsx',encoding='utf-8')

### => 띄어쓰기가 어느정도 처리되었으나, 완전하지 않음. 맞춤법 조정에 의해 '밀키트'가 '밀키스'가 되는 경우 존재.